In [1]:
import os
import subprocess
import sys
os.environ["TOKENIZERS_PARALLELISM"] = "false"

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Instalação das dependências necessárias
dependencies = [
    "sentence-transformers",
    "pymilvus",
    "openai==0.28",
    "langchain_community",
    "minio",
    "pymupdf",
    "Pillow",
    "pytesseract",
    "pandas",
    "langchain-milvus",
    "opencv-python"
]

for dep in dependencies:
    install(dep)

import fitz  # PyMuPDF
import re
import openai
from minio import Minio
from minio.error import S3Error
from sentence_transformers import SentenceTransformer
from langchain_milvus import Milvus
from PIL import Image
import io
import pytesseract
from pytesseract import Output
import pandas as pd
import logging
import urllib3
import cv2

# Configurações do MinIO
AWS_S3_ENDPOINT = "minio-api-minio.apps.rosa.poc-ai.9ewf.p3.openshiftapps.com"
AWS_ACCESS_KEY_ID = ""
AWS_SECRET_ACCESS_KEY = ""
AWS_S3_BUCKET = ""

# Configure logging
logging.basicConfig(level=logging.DEBUG)

# Create an HTTP client with timeouts
http_client = urllib3.PoolManager(
    timeout=urllib3.Timeout(connect=5.0, read=10.0)
)

# Initialize the MinIO client with the custom HTTP client
client = Minio(
    AWS_S3_ENDPOINT,
    access_key=AWS_ACCESS_KEY_ID,
    secret_key=AWS_SECRET_ACCESS_KEY,
    secure=True,
    http_client=http_client
)

# Verifica se o script recebeu o identificador de arquivo como uma variável de ambiente
#file_identifier = os.getenv('file_identifier')
file_identifier = "joao"
if not file_identifier:
    print("Error: file_identifier environment variable is not set.")
    sys.exit(1)
else:
    print(f"file_identifier: {file_identifier}")

# Baixar o arquivo PDF do MinIO
object_name = f"{file_identifier}.pdf"
file_path = f"./{file_identifier}.pdf"
output_pdf_path = f"./{file_identifier}_no_watermark.pdf"

try:
    client.fget_object(AWS_S3_BUCKET, object_name, file_path)
    print(f"'{object_name}' is successfully downloaded to '{file_path}'.")
except S3Error as e:
    print("Error occurred: ", e)
    sys.exit(1)

# Função para remover a marca d'água do PDF
def remove_watermark_advanced(pdf_path, output_path):
    doc = fitz.open(pdf_path)

    for page in doc:
        image_list = page.get_images(full=True)
        for img in image_list:
            xref = img[0]
            page.delete_image(xref)

        annots = page.annots()
        if annots:
            for annot in annots:
                annot_info = annot.info
                if "Watermark" in annot_info.get("title", ""):
                    annot.set_flags(fitz.ANNOT_HIDDEN)

        page.apply_redactions()

    doc.save(output_path)
    print(f"Watermark removed: {output_path}")

# Remover a marca d'água do PDF
remove_watermark_advanced(file_path, output_pdf_path)

# Função para extrair texto do PDF preservando a estrutura dos blocos
def extract_text_from_pdf_with_blocks(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("blocks")
        # Ordenar os blocos pela posição vertical
        blocks = sorted(blocks, key=lambda b: (b[1], b[0]))
        page_text = ""
        for block in blocks:
            block_text = block[4].strip()
            if block_text:
                page_text += block_text + "\n"
        text += page_text + "\n"
    return text

# Função para extrair tabelas do PDF
def extract_tables_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    all_tables = []

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("blocks")
        # Filtrar apenas blocos de texto relevantes
        blocks = [b for b in blocks if b[4].strip()]
        # Criar DataFrame com informações dos blocos
        df = pd.DataFrame(blocks, columns=['x0', 'y0', 'x1', 'y1', 'text', 'block_no', 'block_type'])
        # Ordenar por posição vertical
        df = df.sort_values(by=['y0', 'x0'])
        # Agrupar linhas com base na posição vertical
        lines = []
        current_line = []
        last_y = None
        for index, row in df.iterrows():
            if last_y is None or abs(row['y0'] - last_y) < 5:  # Tolerância de 5 unidades
                current_line.append(row)
            else:
                lines.append(current_line)
                current_line = [row]
            last_y = row['y0']
        if current_line:
            lines.append(current_line)

        # Construir tabela
        table_data = []
        for line in lines:
            line = sorted(line, key=lambda r: r['x0'])
            line_text = [r['text'].strip() for _, r in pd.DataFrame(line).iterrows()]
            table_data.append(line_text)
        if table_data:
            df_table = pd.DataFrame(table_data)
            all_tables.append(df_table)

    return all_tables

# Função para extrair texto via OCR preservando o layout
def ocr_extract_text_with_layout(pdf_path):
    try:
        import pytesseract
        from pytesseract import Output
        import cv2
        import numpy as np
        from PIL import Image, ImageEnhance
        import fitz

        pdf_document = fitz.open(pdf_path)
        text = ""
        for i in range(len(pdf_document)):
            page = pdf_document[i]
            zoom = 2.0
            mat = fitz.Matrix(zoom, zoom)
            pix = page.get_pixmap(matrix=mat)
            img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            # Pré-processamento da imagem
            imagem_processada = img.convert('L')
            enhancer = ImageEnhance.Contrast(imagem_processada)
            imagem_processada = enhancer.enhance(2)
            imagem_array = np.array(imagem_processada)
            imagem_binaria = cv2.adaptiveThreshold(
                imagem_array,
                255,
                cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                cv2.THRESH_BINARY,
                11,
                2
            )
            # Extrair dados com posições
            ocr_data = pytesseract.image_to_data(
                imagem_binaria,
                lang='por',
                config='--psm 6',
                output_type=Output.DICT
            )
            # Construir DataFrame
            df = pd.DataFrame(ocr_data)
            df = df[df.conf != '-1']
            df = df[df.text != ' ']
            df = df[df.text != '']
            df['left'] = df['left'].astype(int)
            df['top'] = df['top'].astype(int)
            df['width'] = df['width'].astype(int)
            df['height'] = df['height'].astype(int)
            df['conf'] = df['conf'].astype(int)
            # Ordenar por posição vertical
            df = df.sort_values(by=['top', 'left'])
            # Agrupar linhas
            lines = df.groupby('line_num')
            page_text = ""
            for line_num, line in lines:
                line = line.sort_values(by='left')
                line_text = ' '.join(line['text'].tolist())
                page_text += line_text + '\n'
            text += page_text + '\n'
        return text

    except Exception as e:
        print(f"Erro ao tentar realizar OCR no arquivo {pdf_path}: {e}")
        return ""

# Função para tentar extrair texto e fallback para OCR se necessário
def extract_text_with_fallback(pdf_path):
    text = extract_text_from_pdf_with_blocks(pdf_path)
    if not text.strip():
        print("Texto não encontrado com PyMuPDF. Tentando extrair tabelas...")
        tables = extract_tables_from_pdf(pdf_path)
        if tables:
            text = ""
            for df_table in tables:
                text += df_table.to_string(index=False, header=False) + '\n'
        else:
            print("Tabelas não encontradas. Iniciando OCR...")
            text = ocr_extract_text_with_layout(pdf_path)
    return text

# Função para extrair nome do contribuinte
def extract_contributor_name(text):
    patterns = [
        r'NOME:\s*([A-Z\s]+)\s*CPF:',
        r'Nome do Contribuinte:\s*([A-Z\s]+)\s*CPF:',
        r'Nome:\s*([A-Z\s]+)\s*Data de Nascimento:',
        r'Declarante:\s*([A-Z\s]+)\s*CPF:',
        r'Contribuinte:\s*([A-Z\s]+)\s*CPF:',
        r'IDENTIFICAÇÃO DO CONTRIBUINTE\s*Nome:\s*([A-Z\s]+)'
    ]

    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            return match.group(1).strip()

    return None

# Configurações do Milvus
MILVUS_HOST = "172.17.0.2"
MILVUS_PORT = 19530
MILVUS_USERNAME = "root"
MILVUS_PASSWORD = "Milvus"
MILVUS_COLLECTION = "safra_dir"

# Configurações do OpenAI
OPENAI_API_KEY = ""
openai.api_key = OPENAI_API_KEY

# Inicialização do modelo de embedding
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

class EmbeddingFunctionWrapper:
    def __init__(self, model):
        self.model = model

    def embed_query(self, query):
        return self.model.encode(query)

    def embed_documents(self, documents):
        return self.model.encode(documents)

embedding_function = EmbeddingFunctionWrapper(embedding_model)

# Extraindo texto do PDF (com fallback para OCR)
text = extract_text_with_fallback(output_pdf_path)
contributor_name = extract_contributor_name(text)

# Verificação se o nome do contribuinte foi encontrado
if not contributor_name:
    print("Erro: Nome do contribuinte não encontrado. O script será encerrado.")
    sys.exit(1)

# Função para converter valores monetários brasileiros para decimal
def convert_brazilian_currency_to_decimal_only(text):
    def replace_func(match):
        value = match.group(0)
        value = value.replace('.', '')
        value = value.replace(',', '.')
        return value

    text = re.sub(r'\b\d{1,3}(\.\d{3})*,\d{2}\b', replace_func, text)
    return text

# Função para extrair código e grupo do texto diretamente do PDF
def extract_group_code_from_pdf(text):
    pattern = r'GRUPO\s*(\d{2})\s+.*\s+CÓDIGO\s*(\d{2})'
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        return match.group(1), match.group(2)
    return None, None

# Função para preprocessar o texto extraído
def preprocess_text_with_group_code(text):
    text = convert_brazilian_currency_to_decimal_only(text)
    group, code = extract_group_code_from_pdf(text)
    return text, group, code

# Função para dividir o texto
def split_text(text, max_length=60000):
    words = text.split()
    parts = []
    current_part = []

    current_length = 0
    for word in words:
        if current_length + len(word) + 1 <= max_length:
            current_part.append(word)
            current_length += len(word) + 1
        else:
            parts.append(" ".join(current_part))
            current_part = [word]
            current_length = len(word) + 1

    if current_part:
        parts.append(" ".join(current_part))

    return parts

# Função para armazenar partes do texto no Milvus
def store_text_parts_in_milvus(text_parts, pdf_file):
    for i, part in enumerate(text_parts):
        preprocessed_text, group, code = preprocess_text_with_group_code(part)
        metadata = {"source": pdf_file, "part": i, "group": group, "code": code}
        store.add_texts([preprocessed_text], metadatas=[metadata])

# Inicialização do Milvus
store = Milvus(
    embedding_function=embedding_function,
    connection_args={"host": MILVUS_HOST, "port": 19530, "user": MILVUS_USERNAME, "password": MILVUS_PASSWORD},
    collection_name=MILVUS_COLLECTION,
    metadata_field="metadata",
    text_field="page_content",
    drop_old=False,
    auto_id=True
)

# Processamento do PDF
pdf_folder_path = './'
for pdf_file in os.listdir(pdf_folder_path):
    if pdf_file.endswith('.pdf') and pdf_file == f"{file_identifier}_no_watermark.pdf":
        pdf_path = os.path.join(pdf_folder_path, pdf_file)
        text = extract_text_with_fallback(pdf_path)
        contributor_name = extract_contributor_name(text)
        if not contributor_name:
            print("Erro: Nome do contribuinte não encontrado. O script será encerrado.")
            sys.exit(1)
        text_parts = split_text(text)
        store_text_parts_in_milvus(text_parts, pdf_file)

print("PDFs processed and stored in Milvus successfully.")

print(f"Contributor name: {contributor_name}")

# Função para realizar a consulta no Milvus e gerar o XML
def query_information(query, contributor_name):
    documents = store.search(query=query, k=1, search_type="similarity")

    context_parts = []
    for doc in documents:
        context = doc.page_content
        group, code = extract_group_code_from_pdf(context)
        if group and code:
            context_parts.append(f"<GRUPO: {group} CÓDIGO: {code}>\n{context}")
        else:
            context_parts.append(context)

    context_combined = "\n".join(context_parts)

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {
                "role": "system",
                "content": "Você é um assistente financeiro avançado, experiente com profundo conhecimento em declaração de imposto de renda. Seu único objetivo é extrair informações do contexto fornecido e gerar respostas no formato XML. NUNCA interrompa uma resposta devido ao seu tamanho. Crie o XML com TODAS as informações pertinentes à sessão de bens e direitos, respeitando TODOS seus atributos e todos seus detalhes."
            },
            {
                "role": "user",
                "content": f"{context_combined}\n\nQuais são todos os bens e direitos, suas informações e seus detalhes, declarados por {contributor_name}? O resultado deve ser apresentado exclusivamente em XML com TODAS as características e detalhes de cada um dos bens, conforme exemplos abaixo:\n\n\
<?xml version=\"1.0\" ?>\n\
<SECTION Name=\"DECLARACAO DE BENS E DIREITOS\">\n\
    <TABLE>\n\
        <ROW No=\"1\">\n\
            <Field Name=\"GRUPO\" Value=\"01\"/>\n\
            <Field Name=\"CODIGO\" Value=\"01\"/>\n\
            <Field Name=\"DISCRIMINACAO\" Value=\"UT QUIS ALIQUAM LEO. DONEC ALIQUA\"/>\n\
            <Field Name=\"SITUACAOANTERIOR\" Value=\"23.445,00\"/>\n\
            <Field Name=\"SITUACAOATUAL\" Value=\"342.342,00\"/>\n\
            <Field Name=\"InscricaoMunicipal(IPTU)\" Value=\"23423424\"/>\n\
            <Field Name=\"Logradouro\" Value=\"RUA QUALQUER\"/>\n\
            <Field Name=\"Numero\" Value=\"89\"/>\n\
            <Field Name=\"Complemento\" Value=\"COMPLEM 2\"/>\n\
            <Field Name=\"Bairro\" Value=\"BRASILIA\"/>\n\
            <Field Name=\"Municipio\" Value=\"BRASÍLIA\"/>\n\
            <Field Name=\"UF\" Value=\"DF\"/>\n\
            <Field Name=\"CEP\" Value=\"1321587\"/>\n\
            <Field Name=\"AreaTotal\" Value=\"345,0 m²\"/>\n\
            <Field Name=\"DatadeAquisicao\" Value=\"12/12/1993\"/>\n\
            <Field Name=\"RegistradonoCartorio\" Value=\"Sim\"/>\n\
            <Field Name=\"NomeCartorio\" Value=\"CARTORIO DE SÇNJJKLCDF ASLK SAKÇK SAÇKLJ SAÇLKS\"/>\n\
            <Field Name=\"Matricula\" Value=\"2344234\"/>\n\
        </ROW>\n\
        <!-- Mais exemplos conforme necessário -->\n\
    </TABLE>\n\
</SECTION>"
            }
        ]
    )

    return response['choices'][0]['message']['content']

# Função principal
def main():
    if contributor_name:
        text = extract_text_with_fallback(output_pdf_path)
        query = f"Quais são todos os bens e direitos suas informações, seus atributos e detalhes, declarados por {contributor_name}?"
        result = query_information(query, contributor_name)

        # Remover formatação de código se existir
        if result.startswith("```xml"):
            result = result[6:]  # Remove "```xml" do início
        if result.endswith("```"):
            result = result[:-3]  # Remove "```" do final

        # Remove espaços desnecessários nas extremidades
        result = result.strip()

        file_name = f'./{file_identifier}.xml'

        with open(file_name, 'w', encoding='utf-8') as file:
            file.write(result)

        print(f'File {file_name} has been successfully saved.')

        # Upload do arquivo XML para o MinIO
        file_path = f"./{file_identifier}.xml"
        object_name = f"{file_identifier}.xml"
        bucket_name = 'irpf-xml'
        try:
            client.fput_object(bucket_name, object_name, file_path)
            print(f"'{object_name}' is successfully uploaded as object to bucket '{bucket_name}'.")
        except S3Error as e:
            print("Error occurred: ", e)
    else:
        print("Erro: Nome do contribuinte não encontrado. O script será encerrado.")
        sys.exit(1)

# Executar o script principal
if __name__ == "__main__":
    main()


/Users/wrosalem/Documents/0x0-Coding/GitHub/rag-architecture-demo/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): minio-api-minio.apps.rosa.poc-ai.9ewf.p3.openshiftapps.com:443


file_identifier: joao


DEBUG:urllib3.connectionpool:https://minio-api-minio.apps.rosa.poc-ai.9ewf.p3.openshiftapps.com:443 "GET /irpf-files?location= HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://minio-api-minio.apps.rosa.poc-ai.9ewf.p3.openshiftapps.com:443 "HEAD /irpf-files/joao.pdf HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://minio-api-minio.apps.rosa.poc-ai.9ewf.p3.openshiftapps.com:443 "GET /irpf-files/joao.pdf HTTP/11" 200 1724267
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443


'joao.pdf' is successfully downloaded to './joao.pdf'.
Watermark removed: ./joao_no_watermark.pdf


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/config_sentence_transformers.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/README.md HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/sentence_bert_config.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/config.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/ma

...

PDFs processed and stored in Milvus successfully.
Contributor name: JOAO CARLOS MARCO
Texto não encontrado com PyMuPDF. Tentando extrair tabelas...
Tabelas não encontradas. Iniciando OCR...


DEBUG:pytesseract:['tesseract', '/var/folders/19/hckvg9wd3mn6f2_44b7bzy6m0000gn/T/tess_uje9xd68_input.PNG', '/var/folders/19/hckvg9wd3mn6f2_44b7bzy6m0000gn/T/tess_uje9xd68', '-l', 'por', '-c', 'tessedit_create_tsv=1', '--psm', '6']
DEBUG:pytesseract:['tesseract', '/var/folders/19/hckvg9wd3mn6f2_44b7bzy6m0000gn/T/tess_46huzmp3_input.PNG', '/var/folders/19/hckvg9wd3mn6f2_44b7bzy6m0000gn/T/tess_46huzmp3', '-l', 'por', '-c', 'tessedit_create_tsv=1', '--psm', '6']
DEBUG:pytesseract:['tesseract', '/var/folders/19/hckvg9wd3mn6f2_44b7bzy6m0000gn/T/tess_f8fbz182_input.PNG', '/var/folders/19/hckvg9wd3mn6f2_44b7bzy6m0000gn/T/tess_f8fbz182', '-l', 'por', '-c', 'tessedit_create_tsv=1', '--psm', '6']
DEBUG:pytesseract:['tesseract', '/var/folders/19/hckvg9wd3mn6f2_44b7bzy6m0000gn/T/tess_73wc6cny_input.PNG', '/var/folders/19/hckvg9wd3mn6f2_44b7bzy6m0000gn/T/tess_73wc6cny', '-l', 'por', '-c', 'tessedit_create_tsv=1', '--psm', '6']
DEBUG:pytesseract:['tesseract', '/var/folders/19/hckvg9wd3mn6f2_44b7bzy6

File ./joao.xml has been successfully saved.


DEBUG:urllib3.connectionpool:https://minio-api-minio.apps.rosa.poc-ai.9ewf.p3.openshiftapps.com:443 "PUT /irpf-xml/joao.xml HTTP/11" 200 0


'joao.xml' is successfully uploaded as object to bucket 'irpf-xml'.
